In [1]:
!pip install pyspark spark-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.8/620.8 kB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=222142c64966f9a904c0087a1accd1ed3690977ea36e2d99bd9ca243d3e81d51
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType

spark = sparknlp.start()
spark

In [5]:
text = ['Peter Parker is a nice lad and lives in New York']

df = spark.createDataFrame(text, StringType()).toDF('text')
df.show(truncate=False)

+------------------------------------------------+
|text                                            |
+------------------------------------------------+
|Peter Parker is a nice lad and lives in New York|
+------------------------------------------------+



In [7]:
documenter = DocumentAssembler()\
    .setInputCol('text')\
    .setOutputCol('document')

tokenizer = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

word_embeddings = WordEmbeddingsModel.pretrained()\
    .setInputCols(['document', 'token'])\
    .setOutputCol('embeddings')

ner_tagger = NerDLModel.pretrained("ner_dl", "en")\
    .setInputCols(['document', 'token', 'embeddings'])\
    .setOutputCol('ner')

ner_converter = NerConverter()\
    .setInputCols(['document', 'token', 'ner'])\
    .setOutputCol('ner_chunk')

chunk_tokenizer = ChunkTokenizer() \
    .setInputCols(["ner_chunk"]) \
    .setOutputCol("chunk_token")


pipeline = Pipeline(stages=[
    documenter,
    tokenizer,
    word_embeddings,
    ner_tagger,
    ner_converter,
    chunk_tokenizer
])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [9]:
model = pipeline.fit(df).transform(df)
model.selectExpr("ner_chunk.result as ner_chunk", "chunk_token.result as chunk_token").show(truncate=False)

+------------------------+--------------------------+
|ner_chunk               |chunk_token               |
+------------------------+--------------------------+
|[Peter Parker, New York]|[Peter, Parker, New, York]|
+------------------------+--------------------------+



In [10]:
documenter = DocumentAssembler()\
    .setInputCol('text')\
    .setOutputCol('document')

tokenizer = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

word_embeddings = WordEmbeddingsModel.pretrained()\
    .setInputCols(['document', 'token'])\
    .setOutputCol('embeddings')

ner_tagger = NerDLModel.pretrained("ner_dl", "en")\
    .setInputCols(['document', 'token', 'embeddings'])\
    .setOutputCol('ner')

ner_converter = NerConverter()\
    .setInputCols(['document', 'token', 'ner'])\
    .setOutputCol('ner_chunk')

chunk_tokenizer = ChunkTokenizer() \
    .setInputCols(["ner_chunk"]) \
    .setOutputCol("chunk_token")


pipeline = Pipeline(stages=[
    documenter,
    tokenizer,
    word_embeddings,
    ner_tagger,
    ner_converter,
    chunk_tokenizer
])

text= ["After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.",
         "Despite the fact that I had saved up 100.000 USD, I still found it difficult to afford a house in the city.",
         "When I was a child, my grandparents would take me on long walks through the countryside, and those memories are still some of my most cherished.",
         "After years of working in the corporate world, I decided to start my own business, and I've never looked back.",
         "The ancient ruins we visited in Rome were so awe-inspiring that I found myself speechless.",
         "Despite the fact that I had been practicing for months, my piano recital was a disaster, and I was humiliated.",
         "When she traveled to Thailand, Mary was amazed by the beauty and serenity of the Buddhist temples.",
         "My grandmother's estate, which had been passed down through several generations of our family, was the site of many cherished family gatherings.",
         "When I was a student, I spent a semester studying abroad in France, and it was one of the most enriching experiences of my life.",
         "After my grandfather passed away, my family and I spent weeks going through his belongings and reminiscing about his life.",
         "The White House is located in Washington, D.C.",
         "I need to exchange dollars for euros before I travel to Europe.",
         "The exchange rate between the pound and the euro is favorable.",
         "As a child, I used to spend my summers at my grandparents' farm, and those days are some of my most cherished memories.",
         "After years of working for a multinational corporation, I decided to quit my job and travel the World.",
         "My great-grandfather fought in World War II, and his stories about the war have been passed down through the generations of our family.",
         "When I was in college, I spent a semester studying abroad in China, and it was an eye-opening experience.",
         "My best friend and I have been planning a trip to South America for years, and we're finally going to make it happen this summer.",
         "My parents own a small motel in a charming seaside town, and they've been running it for over 20 years.",
         "Johnson and I decided to take a road trip across the United States during the summer after we graduated from college."]

df = spark.createDataFrame(text, StringType()).toDF('text')
df.show(truncate=False)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]
+--------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
|After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.|
|Despite the fact that I had saved up 100.000 USD, I still found it difficult to afford a house in the city.                                       |
|When I was a child, my grandparents would take me on long walks t

In [21]:
pipeline = Pipeline(stages=[
    documenter,
    tokenizer,
    word_embeddings,
    ner_tagger,
    ner_converter,
    chunk_tokenizer
])

model = pipeline.fit(df).transform(df)
model.selectExpr("embeddings.result as embeddings", "ner.result as ner", "ner_chunk.result as ner_chunk", "chunk_token.result as chunk_token").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------+-------------------------------+
|embeddings                                                                                                                                                                       |ner                                                                                       |ner_chunk                     |chunk_token                    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------+-------------------------------

In [43]:
text = ['Peter Parker bookad a ticket for the concert. Ticket price is 100$ dollars']
data_set = spark.createDataFrame(text, StringType()).toDF("text")
data_set.show(truncate=False)

+--------------------------------------------------------------------------+
|text                                                                      |
+--------------------------------------------------------------------------+
|Peter Parker bookad a ticket for the concert. Ticket price is 100$ dollars|
+--------------------------------------------------------------------------+



In [45]:
documenter = DocumentAssembler()\
    .setInputCol('text')\
    .setOutputCol('document')

tokenizer = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

token_classifier = RoBertaForTokenClassification.pretrained("roberta_base_token_classifier_ontonotes", "en")\
    .setInputCols(['document', 'token'])\
    .setOutputCol('ner')\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(512)

ner_converter = NerConverter()\
    .setInputCols(['document', 'token', 'ner'])\
    .setOutputCol('entities')

chunk_tokenizer = ChunkTokenizer() \
    .setInputCols(["entities"]) \
    .setOutputCol("chunk_token")\
    .setInfixPatterns(["(\b\d{3}\b)"])

pipeline = Pipeline(stages=[
    documenter,
    tokenizer,
    token_classifier,
    ner_converter,
    chunk_tokenizer
])

roberta_base_token_classifier_ontonotes download started this may take some time.
Approximate size to download 434.7 MB
[OK!]


In [46]:
model = pipeline.fit(data_set).transform(data_set)
model.selectExpr("ner.result as ner", "entities.result as entities", "chunk_token.result as chunk_token").show(truncate=False)

+-------------------------------------------------+--------+-----------+
|ner                                              |entities|chunk_token|
+-------------------------------------------------+--------+-----------+
|[B-PERSON, O, O, O, O, O, O, O, O, O, O, O, O, O]|[Peter] |[Peter]    |
+-------------------------------------------------+--------+-----------+



In [47]:

#Defining exceptions
exceptions= """Peter Parker
James Murphy
Lucas Nelson
"""

#open text file
text_file = open("exceptions.txt", "w")

#write string to file
text_file.write(exceptions)

#close file
text_file.close()

In [51]:

chunk_tokenizer= ChunkTokenizer() \
     .setInputCols(["entities"]) \
     .setOutputCol("chunk_token") \
     .setExceptionsPath('exceptions.txt') \


pipeline = Pipeline(stages=[
    documenter,
    tokenizer,
    token_classifier,
    ner_converter,
    chunk_tokenizer
])

text = ['Peter Parker bookad a ticket for the concert. Ticket price is 100$ dollars']
data_set = spark.createDataFrame(text, StringType()).toDF("text")


result = pipeline.fit(data_set).transform(data_set)
result.selectExpr("entities.result as ner_chunk" , "chunk_token.result as chunk_token").show(truncate=False)

+---------+-----------+
|ner_chunk|chunk_token|
+---------+-----------+
|[Peter]  |[Peter]    |
+---------+-----------+

